In [14]:
import heapq
import random

import geopandas as gpd
import networkx as nx
import osmnx as ox
import pandas as pd
from tqdm.notebook import tqdm

In [15]:
MIN_GRADIENT = 0.08  # Minimum average gradient (e.g., 5%)
MIN_LENGTH = 2000     # Minimum length of the route (in meters)
MAX_DEPTH = 500      # Maximum depth for BFS to avoid excessive computation
MAX_FLAT_SECTION_LENGTH = 50  # Maximum length of sections below gradient threshold (in meters)
MIN_SECTION_GRADIENT = 0.03    # Minimum gradient for each section (e.g., 3%)

In [16]:
def find_candidate_routes(graph, min_gradient, min_length, max_depth):
    candidate_routes = []

    # using a priority queue to prioritize paths with longer lengths
    queue = []
    for start_node in tqdm(graph.nodes, desc="Initializing Queue"):
        heapq.heappush(queue, (-0, start_node, [start_node], 0, 0, 0))  # (negative length, current_node, path, length, elevation_gain, flat_section_length)
    
    while queue:
        neg_length, current_node, path, length, elevation_gain, flat_section_length = heapq.heappop(queue)
        current_depth = len(path)

        if current_depth > max_depth:
            continue

        for neighbor in graph.neighbors(current_node):
            if neighbor in path:
                continue

            edge_data = graph.get_edge_data(current_node, neighbor, 0)
            edge_length = edge_data.get('length', 0)
            edge_gradient = edge_data.get('grade', 0)

            elevation_diff = edge_length * edge_gradient  # calculated elevation gain (can be negative)

            new_length = length + edge_length
            new_elevation_gain = elevation_gain + elevation_diff
            new_avg_gradient = new_elevation_gain / new_length if new_length > 0 else 0

            # update flat/downhill section length if edge gradient is below MIN_SECTION_GRADIENT
            if edge_gradient >= MIN_SECTION_GRADIENT:
                new_flat_section_length = 0
            else:
                new_flat_section_length = flat_section_length + edge_length

            if new_flat_section_length > MAX_FLAT_SECTION_LENGTH:
                continue  # exceeds allowed length of flat/downhill sections

            new_path = path + [neighbor]

            # add path to candidate_routes if it meets constraints
            if new_length >= min_length and new_avg_gradient >= min_gradient:
                candidate_routes.append((new_path, new_length, new_avg_gradient))

            # continue expanding the path, prioritizing by negative length to maximize route length
            heapq.heappush(queue, (-(new_length), neighbor, new_path, new_length, new_elevation_gain, new_flat_section_length))

    return candidate_routes


def assign_routes(candidate_routes):
    # sort routes by length (longest first)
    candidate_routes.sort(key=lambda x: x[1], reverse=True)

    assigned_routes = []
    used_edges = set()

    for path, length, avg_gradient in candidate_routes:
        # check if any edge in the path has already been used
        route_edges = [(u, v) for u, v in zip(path[:-1], path[1:])]
        if any(edge in used_edges or (edge[1], edge[0]) in used_edges for edge in route_edges):
            continue  # skip this route as it overlaps with an assigned route

        # assign the route and mark its edges as used
        assigned_routes.append((path, length, avg_gradient))
        for edge in route_edges:
            used_edges.add(edge)
            used_edges.add((edge[1], edge[0]))  # add both directions if the graph is undirected

    return assigned_routes

In [17]:
# load graph from file
graph = ox.load_graphml("../data/places/asheville.graphml")

In [18]:
# find all candidate routes
candidate_routes = find_candidate_routes(graph, MIN_GRADIENT, MIN_LENGTH, MAX_DEPTH)
print(f"Found {len(candidate_routes)} candidate routes.")

Initializing Queue:   0%|          | 0/24635 [00:00<?, ?it/s]

Found 1578 candidate routes.


In [19]:
# dedupe routes (assign edges to the longest routes first)
assigned_routes = assign_routes(candidate_routes)
print(f"Assigned {len(assigned_routes)} routes after removing overlaps.")

Assigned 7 routes after removing overlaps.


In [20]:
# print route information
for idx, (path, length, gradient) in enumerate(assigned_routes):
    street_names = set()
    for u, v in zip(path[:-1], path[1:]):
        edge_data = graph.get_edge_data(u, v, 0)
        street_name = edge_data.get('name', 'Unnamed Street')
        street_names.add(street_name)
    print(f"Route {idx + 1}: Length = {length:.1f} m, Avg Gradient = {gradient:.2%}, Streets = {', '.join(street_names)}\n")

Route 1: Length = 3510.9 m, Avg Gradient = 8.01%, Streets = Westhaven Drive, Unnamed Street, Charlotte Street, Blue Briar Road, Town Mountain Road, Cherokee Road, Sunset Summit

Route 2: Length = 3339.6 m, Avg Gradient = 10.39%, Streets = Cowan Cove Road, Spivey Mountain Road

Route 3: Length = 3143.0 m, Avg Gradient = 9.52%, Streets = Windsor Road, Unnamed Street, Altamont View, North Merrimon Avenue, Senator Reynolds Road

Route 4: Length = 2970.7 m, Avg Gradient = 8.39%, Streets = Unnamed Street, Lynn Cove Road

Route 5: Length = 2699.2 m, Avg Gradient = 8.18%, Streets = Upper Herron Cove Road, Unnamed Street, Elk Mountain Scenic Highway, Elk Ridge Drive

Route 6: Length = 2370.3 m, Avg Gradient = 8.94%, Streets = Unnamed Street, Baird Cove Road, Versant Drive, Ventana Drive

Route 7: Length = 2018.8 m, Avg Gradient = 8.13%, Streets = Furman Avenue, Westview Road, Clayton Street, Unnamed Street, Oak Park Road, Charlotte Street, Town Mountain Road, Arlington Street



In [21]:
# visualize routes on map
all_routes_gdfs = []
for idx, (path, _, _) in enumerate(assigned_routes):
    route_edges = [(u, v, 0) for u, v in zip(path[:-1], path[1:])]
    route_graph = graph.edge_subgraph(route_edges)
    route_gdf = ox.convert.graph_to_gdfs(route_graph, nodes=False)
    route_gdf["color"] = f"#{random.randint(0, 255):02x}{random.randint(0, 255):02x}{random.randint(0, 255):02x}"
    all_routes_gdfs.append(route_gdf)

all_routes_gdf = gpd.GeoDataFrame(pd.concat(all_routes_gdfs, ignore_index=True))
all_routes_gdf.explore(color=all_routes_gdf["color"], alpha=0.6)